In [ ]:
# analysis micore environment in PETRE
import pandas as pd
import os
import sys
import argparse
def main(argv):
    #! init
    parser = argparse.ArgumentParser(description="")
    parser.add_argument("-g", "--gene", required=True, help="The gene file")
    parser.add_argument("-c", "--chip", required=True, help="The chip file")
    parser.add_argument("-o", "--output", required=True, help='The output file')
    parser.add_argument("-l", "--length", required='True', help="The length of region to be extend")
    parser.add_argument("-n", "--number", required='True', help="The number of splits for the region")
    parser.add_argument("-cn", "--chrom_name", required='True', help="The number of chromsome")
    args = parser.parse_args()
    genef = args.gene
    chipf = args.chip
    L = int(args.length)
    N = int(args.number)
    cn = args.chrom_name
    outputf = args.output
    #! analysis
    #* 1、region extend
    df_gene = pd.read_csv(genef, sep="\t")
    df_gene = region_extend(df_gene, L)
    df_chip = pd.read_csv(chipf, sep="\t")
    df_result = pd.DataFrame()
    #* single chrom 
    #for c in ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']:
    for c in [cn]:
        df_gene_chr = df_gene.query("chrom==@c").copy()
        df_chip_chr = df_chip.query("chrom==@c").copy()
        print(df_gene_chr)
        print(df_chip_chr)
        #* 2、region chip modified 
        df_result = region_chip(df_gene_chr, df_chip_chr, L, N)
    df_result.to_csv(outputf)
# extend the region
def region_extend(df, length):
    df['s'] = df['start'] - length
    df['e'] = df['end'] + length
    df_return = df[['chrom', 's', 'e', 'GeneId']]
    df_return.columns = ['chrom', 'start', 'end', 'GeneId']
    return df_return
# find chip modified on the region
def region_chip(df_gene, df_chip, length, number):
    for index, row in df_gene.iterrows():
        ## add geneid region
        C, S, E, G = row['chrom'], row['start'], row['end'], row['GeneId']
        average_length = int((E-S)/number)
        gene_region_length = E-S
        #print(average_length)
        for i in range(number):
            tags_n = 'E{}'.format(i)
            S_tmp = S
            E_tmp = S + average_length
            df_tmp = df_chip.query("chrom==@C & start <= @E_tmp & end>= @S_tmp").copy()
            tags_number = df_tmp.shape[0]
            df_gene.loc[index, tags_n] = tags_number
            S = E_tmp+1
    return df_gene
if __name__ == "__main__":
    main(sys.argv[1:])

In [ ]:
## 合并PETERs 周围ATAC, H3K4me3, H3K27ac 信号，并且用umap降维聚类
import pandas as pd
chroms = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 
          'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
#tags = ['ATAC', 'H3K27ac', 'H3K4me3']
tags = ['H3K27ac', 'H3K4me3']
df_result = pd.DataFrame()
## 数据合并
for t in tags:
    df_tmp_tags = pd.DataFrame()
    for c in chroms:
        #input_f = "G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/peaks_single/PATREs_peaks/{0}_PETERs_{1}.csv".format(t, c)
        input_f = "G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/peaks_single/PATREs_peaks/hypocotyl_250bp/{0}_PETERs_{1}.csv".format(t, c)
        df = pd.read_csv(input_f)
        df_tmp_tags = pd.concat([df_tmp_tags, df], axis=0)
    if df_result.shape[0]==0:
        df_result = df_tmp_tags.copy()
    else:
        df_result = pd.merge(df_result, df_tmp_tags, how='inner', on=['chrom', 'start', 'end', 'GeneId'])
#umap 降维
def zscore_umap(df):
    import scipy.stats as stats
    from sklearn.preprocessing import StandardScaler
    import umap.umap_ as umap
    #z-score
    df.drop(['chrom', 'start', 'end'], axis=1, inplace=True)
    df.set_index('GeneId', inplace=True)
    df = df.loc[~(df==0).all(axis=1), :]
    print(df.shape[0])
    id_list = df.index.to_list()
    df_numpy = df.to_numpy()
    #df_zscore = stats.zscore(df_numpy, axis=1)
    #df_zscore = pd.DataFrame(df_zscore)
    df_zscore = pd.DataFrame(df_numpy)
    # umap
    reducer = umap.UMAP()
    scaled_penguin_data = StandardScaler().fit_transform(df_zscore)
    embedding = reducer.fit_transform(scaled_penguin_data)
    df_umap = pd.DataFrame(embedding, columns =['umap1', 'umap2'], index=id_list)
    return df_umap
df_umap = zscore_umap(df_result)
df_umap.reset_index(inplace=True)
df_umap.rename(columns={'index':'PATRE_id'}, inplace=True)
df_umap.to_csv('G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/peaks_single/Hypocotyl_PETRE_peaks_single_250bp.csv', index=False)



In [41]:
## 给umap 降维的结果也打上标签
import pandas as pd
df_umap = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/peaks_single/Hypocotyl_PETRE_peaks_single_250bp.csv")
df_patre = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/peaks_single/hypocotyl_PATRE_tags.csv", names=['chrom', 'start', 'end', 'PATRE_id', 'tags'])
df_result = pd.merge(df_umap, df_patre, how='outer', on=['PATRE_id'])
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/peaks_single/hypcotyl_PETRE_peaks_single_250bp_tags.csv", index=False)

In [ ]:
## 合并PETERs 周围ATAC, H3K4me3, H3K27ac 信号，并且用umap降维聚类
import pandas as pd
import numpy as np

def add_tags(df):
    df = df.copy()
    df[df<=0] = 0
    df['total'] = df.select_dtypes(np.number).gt(0.1).sum(axis=1)
    df['tags'] = df['total'].apply(lambda x: 'T' if x>=20 else 'F')
    df.drop('total', inplace=True, axis=1)
    return df

def get_tags(df1, df2):
    df1 = df1.copy()
    df2 = df2.copy()
    df1.set_index('PATRE_Id', inplace=True)
    df2.set_index('PATRE_Id', inplace=True)
    df1 = add_tags(df1)
    df2 = add_tags(df2)
    df1.reset_index(inplace=True)
    df2.reset_index( inplace=True)
    print(df1.head())
    #print(df1)
    df = pd.merge(df1, df2, how='inner', on=['PATRE_Id'])
    df['tags'] = df['tags_x'].map(str) + "-" + df['tags_y'].map(str)
    df_tags = df[['PATRE_Id', 'tags']].copy()
    return df_tags


#umap 降维
def zscore_umap(df):
    import scipy.stats as stats
    from sklearn.preprocessing import StandardScaler
    import umap.umap_ as umap
    #z-score
    #df.drop(['chrom', 'start', 'end'], axis=1, inplace=True)
    df.set_index('PATRE_Id', inplace=True)
    df = df.loc[~(df==0).all(axis=1), :]
    df.fillna(0, inplace=True)
    #df[df<=0] = 0
    #print(df.shape[0])
    id_list = df.index.to_list()
    #print(df.shape)
    df_numpy = df.to_numpy()
    df_zscore = stats.zscore(df_numpy, axis=1)
    df_zscore = pd.DataFrame(df_zscore)
    #df_zscore = pd.DataFrame(df_numpy)
    df_zscore.fillna(0, inplace=True)
    # umap
    reducer = umap.UMAP()
    scaled_penguin_data = StandardScaler().fit_transform(df_zscore)
    embedding = reducer.fit_transform(scaled_penguin_data)
    df_umap = pd.DataFrame(embedding, columns =['umap1', 'umap2'], index=id_list)
    return df_umap


## 数据合并
output_f = "G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_UMAP.csv"
df_all = pd.DataFrame()
for t in ['hypocotyl', 'ovule', 'stigma', 'radicle']:
    for i in ['ATAC', 'H3K27ac', 'H3K4me3']:
        input_f = "G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/{0}_{1}_1K_100bp.csv".format(t, i)
        df_tmp = pd.read_csv(input_f)
        if df_all.shape[0] == 0:
            df_all = df_tmp.copy()
        else:
            df_all = pd.merge(df_all, df_tmp, how='inner', on=['PATRE_Id'])
df_annot = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_annotation.bed", sep=" ")
df_annot['promoter'] = df_annot['annotation'].apply(lambda x: -100 if 'Distal' in str(x) else 100)
df_annot = df_annot[['V4', 'promoter']].copy()
df_annot.columns = ['PATRE_Id', 'promoter']
#print(df_annot)
df_result = pd.merge(df_all, df_annot, how='left', on=['PATRE_Id'])
#df_result = df_all.copy()
#print(df_result.head())
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/a.csv", index=False)

df_H3K27ac = pd.DataFrame()
df_H3K4me3 = pd.DataFrame()
## H3K27ac
for i in ['hypocotyl', 'ovule', 'stigma', 'radicle']:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/{0}_H3K27ac_1K_100bp.csv".format(i)
    df_tissue = pd.read_csv(input_f)
    #print(df_tissue)
    if(df_H3K27ac.shape[0]==0):
        df_H3K27ac = df_tissue.copy()
    else:
        #print(df_H3K27ac.head())
        #print(df_tissue.head())
        df_H3K27ac = pd.merge(df_H3K27ac, df_tissue, how='inner', on=['PATRE_Id'])
df_H3K4me3 = pd.DataFrame()
for i in ['hypocotyl', 'ovule', 'stigma', 'radicle']:
    input_f = "G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/{0}_H3K4me3_1K_100bp.csv".format(i)
    df_tissue = pd.read_csv(input_f)
    if(df_H3K4me3.shape[0]==0):
        df_H3K4me3 = df_tissue.copy()
    else:
        df_H3K4me3 = pd.merge(df_H3K4me3, df_tissue, how='inner', on=['PATRE_Id'])

df_PATRE_Tag = get_tags(df_H3K4me3, df_H3K27ac)

df_umap = zscore_umap(df_result)
df_umap.reset_index(inplace=True)
df_umap.rename(columns={'index':'PATRE_Id'}, inplace=True)

# ## 打上chromHMM 标签
df_result = pd.merge(df_umap, df_PATRE_Tag, how='left', on =['PATRE_Id'])
old_tags = ['F-F', 'F-T', 'T-F', 'T-T']
#new_tags = ['Non-functional elements', 'Candidate enhancers', 'Candidate promoters', 'Candidate promoters']
new_tags = ['Non-functional elements', 'Candidate enhancers', 'Candidate promoters', 'Candidate promoters']

df_result['tags'].replace(old_tags, new_tags, inplace=True)
df_result.to_csv(output_f, index=False)

In [ ]:
## 分析各个组织活性的cis元件
## 根据所鉴定的cis调控元件在其它组织中是否受到loop调控确定是否为组织激活cis调控元件
import pandas as pd
df_cis = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/loops/Cis-element/")

In [64]:
### 对上一步umap的结果进行校正，用PATRE的注释信息来校正，
# 被定义为候选启动子，但是位于远端调控区域，修正为候选增强子
# 被定义为候选增强子，但是位于非远端调控区域，修正为候选启动子
import pandas as pd
df_PATRE = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_UMAP.csv")
df_annot = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_annotation.bed", sep=" ")
df_distal = df_annot.query("annotation.str.contains('Distal')", engine='python')[['V4']].copy()
df_distal['tags1'] = ['Distal']*df_distal.shape[0]
df_distal.columns = ['PATRE_Id', 'tags1']

df_distal.fillna("Clost", inplace=True)
df_PATRE = pd.merge(df_PATRE, df_distal, how='left', on=['PATRE_Id'])
## 更正候选增强子
df_enhencer_correct = df_PATRE.query("tags=='Candidate promoters' & tags1=='Distal'").copy()
df_enhencer_correct['tags'] = ['Candidate enhancers']*df_enhencer_correct.shape[0]
print(df_enhencer_correct.shape[0]) 
## 更正候选启动子
df_promoter_correct = df_PATRE.query("tags=='Candidate enhancers' & tags1!='Distal'").copy()
df_promoter_correct['tags'] = ['Candidate promoters']*df_promoter_correct.shape[0]
print(df_promoter_correct.shape[0])
df_result = pd.concat([df_PATRE, df_enhencer_correct, df_promoter_correct], axis=0)
df_result.drop_duplicates(keep='last', subset=['PATRE_Id'], inplace=True)
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_UMAP_correct.csv", columns=['PATRE_Id', 'umap1', 'umap2', 'tags'], index=False)


7824
5574


In [53]:
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_UMAP_correct.csv")
df['number'] = [1]*df.shape[0]
df_result = df.groupby(by=['tags'], as_index=False)['number'].sum()
print(df_result)

                      tags  number
0      Candidate enhancers   18520
1      Candidate promoters   26561
2  Non-functional elements    9531


In [1]:
## 分析所鉴定的活性启动子所调控的基因与其它基因在表达水平上的差异
import pandas as pd
df_PATRE = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_UMAP_correct.csv")
df_PATRE_promoter = df_PATRE.query("tags=='Candidate promoters'").copy()
df_annot = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_annotation.bed", sep=" ")
df_annot = df_annot[['V4', 'geneId']].copy()
df_annot.columns = ['PATRE_Id', 'GeneId']
df_promoter_annot = pd.merge(df_annot, df_PATRE_promoter, how='inner', on=['PATRE_Id'])[['GeneId', 'tags']].copy()
df_promoter_annot.drop_duplicates(keep='first', inplace=True)
df_gene_TPM = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/RNA_Seq_data/J668_all_Tissue_mean_TPM_long.csv")
df_PATRE_TPM = pd.merge(df_gene_TPM, df_promoter_annot, how='left', on=['GeneId'])
df_PATRE_TPM.fillna("no active promoter", inplace=True)
old_tags = ['Candidate promoters', 'no active promoter']
new_tags = ['A_P', 'N-A_P']
df_PATRE_TPM['tags'].replace(old_tags, new_tags, inplace=True)
df_PATRE_TPM.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_promoter_gene.csv", index=False)

In [8]:
## 分析活性promoter启动子调控基因中管家基因，沉默基因和动态基因的分布
df_gene_type = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/new_RNA_Seq/RNA_Seq_data/J668_gene_type.csv")
df_promoter_tags = df_PATRE_TPM[['GeneId', 'tags']].drop_duplicates(keep='first').copy()
df_tmp = pd.merge(df_gene_type, df_promoter_tags, how='inner', on=['GeneId'])
df_tmp['number'] = [1]*df_tmp.shape[0]
df_result = df_tmp.groupby(by=['sigs', 'tags'], as_index=False)['number'].sum()
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_promoter_gene_type.csv", index=False)

In [83]:
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/hypocotyl/hypocotyl_micro_state.csv")
df_RNA = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/hypocotyl/hypocotyl_peaks_1Kb_RNA_Seq.bed", sep="\t", names=['chrom', 'start', 'end', 'PATRE_Id', 'Reads'])
df_RNA.drop(['chrom', 'start', 'end'], inplace=True, axis=1)
df_result = pd.merge(df, df_RNA, how='left', on=['PATRE_Id'])
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/hypocotyl/hypocotyl_micro_state.csv")

In [10]:
# 将所有组织的chromHMM 合并为一个文件
import pandas as pd
df = pd.read_csv("all_chromHMM_segments.bed", sep="\t", names=['chr', 'start', 'end','leaf', 'hypocotyl', 'cotyledon', 'stigma', 'radicle'])
df_same = df.query("leaf==hypocotyl & hypocotyl==cotypldon & cotyledon==stigma & stigma==radicle").copy()
df_dynamic = pd.concat([df, df_same], axis=0)
df_dynamic.drop_duplicates(keep=False, inplace=True)
df_same['tags'] = df_same['leaf']
for index, row in df_dynamic.iterrows():
    if 'E2' in row:
        t = 'E2'
    elif 'E1' in row and 'E4' in row:
        t = 'E2'
    elif 'E1' in row:
        t = 'E1'
    elif 'E4' in row:
        t = 'E4'
    else:
        t = 'E3'
    df_dynamic

In [ ]:
## 给PATERs打上标签 
# Nutures
# 20222203
# add chromHMM tags for PATERs
import pandas as pd
import sys
paters = sys.argv[1]
chromHMM = sys.argv[2]
output = sys.argv[3]
Chrom = sys.argv[4]
df_paters = pd.read_csv(paters, sep="\t", names=['chrom', 'start', 'end', 'PATRE_ID'])
df_chromHMM = pd.read_csv(chromHMM, sep="\t", names=['chrom', 'start', 'end', 'tags'])
c = Chrom
all_chrom = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13',
             'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
df_paters_c = df_paters.query("chrom==@c").copy()
df_chromHMM_c = df_chromHMM.query("chrom==@c").copy()
for index, row in df_paters_c.iterrows():
    C, S, E = row['chrom'], row['start'], row['end']
    print(C, S, E)
    df_tmp = df_chromHMM_c.query("chrom==@C & start<=@E & end>= @S").copy()
    print(df_tmp)
    all_tags = "-".join(list(set(df_tmp['tags'].tolist())))
    print(all_tags)
    df_paters_c.loc[index, 'tags'] = all_tags
df_paters_c.to_csv(output, index=False)

In [31]:
## 给ATAC 位点打上PATERs 标签
# Nutures
# 202221004
# add chromHMM tags for ATAC site
import pandas as pd
import sys
atac = sys.argv[1]
chromHMM = sys.argv[2]
output = sys.argv[3]
Chrom = sys.argv[4]
df_atac = pd.read_csv(atac, sep="\t", names=['chrom', 'start', 'end', 'ID'])
df_chrom = pd.read_csv(chromHMM, sep="\t", names=['chrom', 'start', 'end', 'Tags'])
df_atac_c = df_atac.query("chrom==@Chrom").copy()
df_chrom_c = df_chrom.query("chrom==@Chrom").copy()
for index, row in df_atac_c.iterrows():
    C, S, E = row['chrom'], row['start'], row['start']
    df_tmp = df_chrom_c.query("chrom==@C & start<=@E & end>= @S").copy()
    if df_tmp.shape[0] == 0:
        all_tags = 'E0'
    else:
        df_tmp.reset_index(drop=True, inplace=True)
        all_tags = df_tmp.loc[0, 'Tags']
    df_atac_c.loc[index, 'Tags'] = all_tags
df_atac_c.to_csv(output, sep="\t", index=False)

In [ ]:
## hypocotyl , ovule, radicle , stigma这几种组织的染色质开放区域置合并起来
df_hypocotyl = pd.read_csv("hypocotyl/hypocotyl_PATRE_around_1K.bed", sep="\t", names=['chrom', 'start', 'end', 'id'])
df_radicle = pd.read_csv("radicle/radicle_PATRE_around_1K.bed", sep="\t", names=['chrom', 'start', 'end', 'id'])
df_ovule = pd.read_csv("ovule/ovule_PATRE_around_1K.bed", sep="\t", names=['chrom', 'start', 'end', 'id'])
df_stigma = pd.read_csv("stigma/radicle_PATRE_around_1K.bed", sep='\t', names=['chrom', 'start', 'end', 'id'])

df_all = pd.concat([df_hypocotyl, df_radicle, df_ovule, df_stigma], axis=0)
df_all.drop_duplicates(keep='first', subset=['chrom', 'start', 'end'], inplace=True)
df_all['id'] = ['PATER']*df_all.shape[0]
df_all['number'] = range(df_all.shape[0])
df_all['id2'] = df_all['id'].map(str) + df_all['number'].map(str)
df_all.to_csv("J668_PATRE.bed", sep="\t", index=False)



In [6]:
## 给PATRE打上标签
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE.bed", sep="\t", names=['chrom', 'start', 'end', 'PATRE_Id'])
df_tags = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_UMAP_correct.csv")
df_result = pd.merge(df, df_tags, how='inner', on=['PATRE_Id'])
df_result.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/ATAC/PETRE/THR/J668_PATRE_tags.bed", sep="\t", index=False, header=False)

In [5]:
## 分为At和Dt两个Cis
import pandas as pd
df = pd.read_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/loops/Cis-element/J668_cis.bed", sep="\t")
df_At = df.query("chrom.str.contains('Ghir_A')", engine='python').copy()
df_Dt = df.query("chrom.str.contains('Ghir_D')", engine='python').copy()
df_At['tags1'] = ['At']*df_At.shape[0]
df_At['tags2'] = range(df_At.shape[0])
df_At['tags'] = df_At['tags1'].map(str) + df_At['tags2'].map(str)

df_Dt['tags1'] = ['Dt']*df_Dt.shape[0]
df_Dt['tags2'] = range(df_Dt.shape[0])
df_Dt['tags'] = df_Dt['tags1'].map(str) + df_Dt['tags2'].map(str)

df_At.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/loops/Cis-element/J668_Cis_At.bed", sep="\t", index=False, columns=['chrom', 'start', 'end', 'tags'], header=False)
df_Dt.to_csv("G:/Billfish/J668_multip_tissue_3D-genome/HiC/loops/Cis-element/J668_Cis_Dt.bed", sep="\t", index=False, columns=['chrom', 'start', 'end', 'tags'], header=False)
